# Question 1

In [388]:
import pandas as pd
from datetime import datetime
from datetime import date
import numpy as np
from tqdm import tqdm
import itertools
from random import randint
from sklearn.utils import shuffle


##  1.1 Data Engineering

In [306]:
df = pd.read_csv('Data/bank_transactions.csv',usecols=['CustomerID','CustomerDOB','CustGender','CustAccountBalance'])

In [307]:
df=df.groupby('CustomerID').first().reset_index()

In [308]:
df['CustomerDOB']= pd.to_datetime(df['CustomerDOB'])

In [309]:
df.drop(df[(df['CustomerDOB'] == '1800-1-1')].index, inplace = True)

In [310]:
df=df.dropna()

In [311]:
def age(birthdate):
    today = date.today()
    one_or_zero = ((today.month, today.day) < (birthdate.month, birthdate.day))
    year_difference = today.year - birthdate.year
    age = year_difference - one_or_zero
    return age

In [312]:
df["CustomerDOB"] = df['CustomerDOB'].apply(age)

In [313]:
df.drop(df[(df['CustomerDOB'] < 16)].index, inplace = True)

In [314]:
df['CustGender']=df['CustGender'].apply(lambda x: 5 if (x=='M') else 6)

In [315]:
def ageClassification(age):
    if age <= 21:
        return 0
    elif age > 21 and age <= 30:
        return 1
    elif age > 30 and age <= 45:
        return 2
    elif age > 45 and age <= 60:
        return 3   

In [316]:
df['CustomerDOB']=df['CustomerDOB'].apply(ageClassification)

In [317]:
df

,CustomerID,CustomerDOB,CustGender,CustAccountBalance
0,C1010011,1,6,32500.73
1,C1010012,1,5,24204.49
2,C1010014,1,6,38377.14
3,C1010018,2,6,496.18
5,C1010028,2,6,296828.37
...,...,...,...,...
884260,C9099836,2,5,133067.23
884261,C9099877,1,5,96063.46
884262,C9099919,1,5,5559.75
884263,C9099941,1,5,35295.92


In [321]:
a = pd.qcut(np.array(df["CustAccountBalance"]), 4)

In [322]:
a = a.unique().dropna()

In [323]:
def balanceClassification(x):
    if x in a[1]:
        return 4
    elif x in a[3]:
        return 7
    elif x in a[0]:
        return 8
    else:
        return 9

In [324]:
df["CustAccountBalance"] = df['CustAccountBalance'].apply(balanceClassification)

In [325]:
df = df.reset_index()


In [326]:
df = df.drop(['index'], axis=1)

In [327]:
mat = np.zeros((10,df.shape[0],))

In [328]:
df_min = pd.DataFrame(mat)

In [329]:
df

,CustomerID,CustomerDOB,CustGender,CustAccountBalance
0,C1010011,1,6,8
1,C1010012,1,5,8
2,C1010014,1,6,8
3,C1010018,2,6,4
4,C1010028,2,6,9
...,...,...,...,...
768518,C9099836,2,5,9
768519,C9099877,1,5,9
768520,C9099919,1,5,7
768521,C9099941,1,5,8


## 1.2 Fingerprint hashing

In [331]:
#Preparing the shingles and the sets
li = [*range(10)]
for i in tqdm(range(len(li))):
    for row in df.index:
        if df.loc[row]['CustomerDOB'] == i:
            df_min.loc[i][row] = 1
        
        if df.loc[row]['CustGender'] == i:
            df_min.loc[i][row] = 1
            
        if df.loc[row]['CustAccountBalance'] == i:
            df_min.loc[i][row] = 1
            


100%|███████████████████████████████████████████| 10/10 [13:25<00:00, 80.58s/it]


In [ ]:
df_min.to_csv('one_hot_encoding.csv', encoding='utf-8')

In [360]:
df_min

,0,1,2,3,4,5,6,7,8,9,...,768513,768514,768515,768516,768517,768518,768519,768520,768521,768522
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
2,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
8,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


In [380]:
#Getting the Signatures for the minhashing
signatures = []
for perm in tqdm(range(21)):
    sig = []
    permDf=shuffle(df_min)
    for col in permDf:
        for row in permDf.index:
            if permDf.loc[row][col]==1:
                sig.append(row)
                break
    signatures.append(sig)



100%|███████████████████████████████████████████| 21/21 [14:51<00:00, 42.46s/it]


In [381]:
signArr = np.array(signatures)

In [386]:
signatureDf = pd.DataFrame(signArr)
signatureDf=signatureDf.T

In [387]:
signatureDf

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,6,8,6,1,8,1,6,6,8,8,...,8,8,1,8,8,1,6,1,8,1
1,1,8,5,5,8,1,5,8,8,5,...,8,5,1,5,8,1,1,1,5,5
2,6,8,6,1,8,1,6,6,8,8,...,8,8,1,8,8,1,6,1,8,1
3,6,2,6,4,6,2,2,6,6,2,...,6,4,6,6,2,4,2,4,2,2
4,6,2,6,9,9,2,2,6,6,2,...,6,2,6,6,9,9,2,9,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768518,2,2,9,5,9,2,5,2,5,5,...,9,2,9,5,9,9,2,5,5,2
768519,1,9,9,5,9,9,5,1,5,5,...,9,9,1,5,9,1,1,1,5,5
768520,1,7,5,5,7,1,5,7,7,5,...,5,5,1,5,7,1,7,1,5,5
768521,1,8,5,5,8,1,5,8,8,5,...,8,5,1,5,8,1,1,1,5,5


In [389]:
def splitSignatures(s,b):
    x = int(len(s)/b)
    bands = []
    for i in range (0,len(s),x):
        bands.append(s[i,i+r])
    return bands